In [1]:
from nltk.probability import FreqDist
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import heapq
import nltk.data
import csv
import numpy as np
import pandas as pd
import math
nltk.download('stopwords')
nltk.download('punkt')



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ines2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ines2\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
#read the lyrics.csv
# the lyrics and scores are empty for new songs, they have to be filled first
df = pd.read_csv('Lyrics.csv',delimiter=";",encoding='latin1')
print(df)

               Song Title                                          Artist  \
0           Feel So Close                                   Calvin Harris   
1       Turn Up the Music                                     Chris Brown   
2    Club Can't Handle Me                          Flo Rida, David Guetta   
3           Rain in Ibiza  Felix Jaehn, The Stickmen Project, Calum Scott   
4                Believer                                 Imagine Dragons   
..                    ...                                             ...   
993                   NaN                                             NaN   
994                   NaN                                             NaN   
995                   NaN                                             NaN   
996                   NaN                                             NaN   
997                   NaN                                             NaN   

     Lyrics    bpm  Push myself score  Have fun score  Stress relief score 

In [8]:
# API Call for lyrics. 
# Don't always run it for all songs, as the API only allows 100 calls per day. 
# Therefore put i on the first line with empty lyrics
i = 0
from lyricsgenius import Genius
token = 'Wbdl4zQaphgb0HpfLCqJuhJWNU2B2m7NDXTh3oN9TE5bmM77K1fS68czEbS_wSHy'
genius = Genius(token)
songs =[]
for artist in df.loc[:,"Artist"]:
    if i>103: #put here the first empty line
        print('Start')
        while True:
            try:
                song = genius.search_song(df.loc[i]['Song Title'],artist)
                break
            except:
                pass
        try:
            df.at[i,'Lyrics'] = song.lyrics
        except:
            df = ddf.drop(i)
    i+= 1
#print(artists[1].lyrics)   

ModuleNotFoundError: No module named 'lyricsgenius'

In [ ]:
# Here the new lyrics are written to the file more_lyric.csv. 
# Either change this to write to lyrics.csv or copy-paste. 
# This was only the test phase so far.
df.to_csv('more_lyrics.csv',sep=';',encoding='utf-8-sig')

In [8]:
#lookup line for finding errors
print(df.iloc[103]['Song Title'])

It Was A Very Good Year


In [12]:
#scores for our categories
#TODO
push_myself ={'push','strive','grind','plod','labor','moil','strain','tug','speed','rocket','opportunities','opportunity','hard','shouting','stopping','force','pain','believer','limit','sky','shit','fuck','unstoppable','pump','adrenaline'}
have_fun = {'fun','happy','fancy','savor','enjoy','good','great','awesome','amazing','nice','sweet','fantastic','sunshine','dancing','alive','dance','party','dj','music','sex','whoop','tickle','clowns'}
stress_relief = {'stress', 'relief','stressless','stressfree','reduction','alright','kind','sleep','breathe','breathing','time','love','kiss','sun','stars','moon','sky'}
relax_focus={'relax','focus','focused','focusing','focussed','focussing','relaxation','recreation','relaxing','relaxed','recreational','unwind','unwinding','de-stressing','soothe','soothing','eased','ease','dream','fairy','paradise','beautiful','mind','butterflies','concentrate','concentration','chill','chilling','chillin','chilled'}
get_motivated ={'motivation','motivating','motivated','motivate','motifing','animate','animating','animated','encouraging','encourage','encouraged','exciting','inspiring','inspire','inspired','inspirational','stirring','exhilarating','uplifting','winning','win','winner','easy','survive','tiger','work','countdown'}

In [13]:
#counters for statistics
push_myself_counter= 0
have_fun_counter = 0
stress_relief_counter = 0
relax_focus_counter = 0
get_motivated_counter = 0

In [4]:

def summerise(input, i):
    scores = [0,0,0,0,0] #push myself, have fun, stress relief, relax/focus, motivation
    
    # (1) Convert paragraphs to sentences
    sentence_detector = nltk.data.load('tokenizers/punkt/english.pickle')
    sentences = sentence_detector.tokenize(input)
    #sentences = nltk.sent_tokenize(input)
    
    # (2) Tokenisation: split sentences to words
    tokenizer = RegexpTokenizer('\w+')
    
    # (3) Preprocessing: lowercase words and remove stopwords
    base_words = [word.lower() for word in tokenizer.tokenize(input)]
    words_fist_step = [word for word in base_words if word not in stopwords.words()]
    
    #TODO
    #processing because of problems with API lyrics. Have a look at them, the problem are the words in [], they distort the statistics
    #This will filter out some common ones, but it is best to use a filter that excludes all words surrounded by square brackets.
    words = [word for word in words_fist_step if word not in ['chorus','lyrics','lyric','guitar solo','solo','verse','verse 1','verse 2','verse 3','verse 4','verse 5','outro','intro','refrain','instrumental','instrumental post-chorus','interlude','bridge','pre chorus','pre-chorus','post-chorus','post chorus','ad lib','ad-lib']]

    # (4) Find Weighted Frequency of Occurence      
    word_frequencies = FreqDist(words)
        
    maximum_frequency = max(word_frequencies.values())

    for word in word_frequencies.keys():
        word_frequencies[word] = (word_frequencies[word]/maximum_frequency)
    #print('---------------------------------------------')
    print(word_frequencies.most_common())
    #print('---------------------------------------------')
    print(word_frequencies.values())
    for word in word_frequencies.keys():
        if(word in push_myself):
            scores[0]+= frequencies(word).values()
        if(word in have_fun):
            scores[1]+= 1    
        if(word in stress_relief):
            scores[2]+= 1    
        if(word in relax_focus):
            scores[3]+= 1   
        if(word in get_motivated):
            scores[4]+= 1   
   # print(summary)
    if scores[0]>0:
        global push_myself_counter
        push_myself_counter+= 1
    if scores[1]>0:
        global have_fun_counter
        have_fun_counter+= 1
    if scores[2]>0:
        global stress_relief_counter
        stress_relief_counter+= 1
    if scores[3]>0:
        global relax_focus_counter
        relax_focus_counter+= 1
    if scores[4]>0:
        global get_motivated_counter
        get_motivated_counter+= 1
    print('Push myself score:'+ str(scores[0]))
    print('Have fun score:'+ str(scores[1]))
    print('Stress relief score:'+ str(scores[2]))
    print('Relax / focus score:'+ str(scores[3]))
    print('Get motivated score:'+ str(scores[4]))
    df.at[i,'Push myself score'] = scores[0]
    df.at[i,'Have fun score'] = scores[1]
    df.at[i,'Stress relief score'] = scores[2]
    df.at[i,'Relax / focus score'] = scores[3]
    df.at[i,'Get motivated score'] = scores[4]
    return scores


In [5]:
#Here the processing happens, the scores re calculated, if the are empty so far
i = 0
for song in df.loc[:,"Lyrics"]:
    print ('******** '+df.loc[i]['Song Title']+' ********')
    if math.isnan(df.loc[i]['Get motivated score']):
        summerise(song, i)
    i+= 1
    #print('*******************************************************')
    print('\n')

******** Feel So Close ********


TypeError: expected string or bytes-like object

In [16]:
# Here the new scores are written to the file more_lyric.csv. 
# Either change this to write to lyrics.csv or copy-paste. 
# This was only the test phase so far.
# (Same as with the lyrics before)
df.to_csv('more_lyrics.csv',sep=';',encoding='utf-8-sig')

In [ ]:
print('Push myself songs: '+str(push_myself_counter))
print('Have fun songs: '+str(have_fun_counter))
print('Stress relief songs: '+str(stress_relief_counter))
print('Relax/Focus songs: '+ str(relax_focus_counter))
print('Get motivated songs: '+str(get_motivated_counter))

In [ ]:
#processing for diagram
t_array = df["bpm"].to_numpy() 
print(t_array)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

n, bins, patches = plt.hist(t_array)
plt.show()

In [20]:
print(t_array.size)

56


In [5]:
#Tutorial for Genius
'''from lyricsgenius import Genius
token = 'Wbdl4zQaphgb0HpfLCqJuhJWNU2B2m7NDXTh3oN9TE5bmM77K1fS68czEbS_wSHy'
genius = Genius(token)
artist = genius.search_artist("Andy Shauf", max_songs=3, sort="title")
print(artist.songs)
# Way 1
song = genius.search_song("To You", artist.name)

# Way 2
# this will search artist.songs first
# and if not found, uses search_song
song = artist.song("To You")

print(song.lyrics)'''


Searching for songs by Andy Shauf...

Song 1: "Alexander All Alone"
"All Of My Love" is not valid. Skipping.
Song 2: "All the Same"
Song 3: "Angela"

Reached user-specified song limit (3).
Done. Found 3 songs.
[Song(id, artist, ...), Song(id, artist, ...), Song(id, artist, ...)]
Searching for "To You" by Andy Shauf...
Done.
Searching for "To You" by Andy Shauf...
Done.
To You LyricsJeremy, can we talk a minute?
I've got some things that I need to
Get off of my chest

I know that we've had a few
And it's far too late
But if I wait, I might never tell you
Can we find somewhere quiet?
Let's go outside
I've got some smokes if you've got a light
It's just that sometimes when I'm by your side
It feels so right
It feels like nothing could go wrong
Does it ever
Feel like that to you?
Does it ever
Feel like that to you?
Oh, I don't know what I mean
It sounded wrong and I'm just tired
Not being weird
I just mean that you're a good friend
It's hard to explain
Just forget I said anything
Get over 

In [35]:
#Test for bpm API, does not work
key= '0bf7106cdcb361896dd1cbf071ff715e'
url_bpm = 'https://api.getsongbpm.comsong/?api_key='+key
url_song = "https://api.getsongbpm.com/song/?api_key="+key+"&id=983pB"
response = requests.get(url_song)
#json_data = json.loads(response.content)
print(response)

<Response [403]>
